# Import / Config

In [1]:
!uv pip install -U psycopg[binary,pool] sqlalchemy pandas dotenv

Using Python 3.12.8 environment at: .py312
Resolved 14 packages in 90ms                                         
Audited 14 packages in 0.06ms


In [25]:
import pandas as pd
import json
from dataclasses import dataclass
from typing import List, Optional
from sqlalchemy import create_engine, text
from dotenv import dotenv_values

c_pg = dotenv_values("cred-pg.txt")

# None = unbegrenzt
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", None)

dummy_json = json.loads('[ { "dummy": "bisher nicht implemenietiert" } ]')

In [3]:
engine = create_engine(
    f'postgresql+psycopg://{c_pg["user"]}:{c_pg["password"]}@{c_pg["host"]}/{c_pg["dbname"]}', 
    connect_args = {
        'options': '-c search_path=ucla', 
        'keepalives_idle': 120
    },
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

# Datenbank

## drop_tables

In [4]:
def drop_tables(engine):
    sql1 = "drop table if exists t3;"
    sql2 = "drop table if exists t2;"
    sql3 = "drop table if exists t1;"
    with engine.connect() as conn:
        conn.execute(text(sql1))
        conn.execute(text(sql2))
        conn.execute(text(sql3) ) 

## create_tables

In [5]:
def create_tables(engine):
    sql1 = """
    create table t1 (
      pk1 integer primary key,
      a varchar,
      b varchar
    )
    """
    sql2 = """
    create table t2 (
      pk2 integer primary key,
      c varchar,
      d varchar,
      fk_t2_t1 integer,
      constraint fk_t2_t1 foreign key (fk_t2_t1) references t1
    )
    """
    sql3 = """
    create table t3 (
      pk3 integer primary key,
      e varchar,
      f varchar,
      fk_t3_t2 integer,
      constraint fk_e3_e2 foreign key (fk_t3_t2) references t2
    )
    """
    with engine.connect() as conn:
        conn.execute(text(sql1))
        conn.execute(text(sql2)) 
        conn.execute(text(sql3))   

## delete_data

In [6]:
def delete_data(engine):
    sql1 = "delete from t3"
    sql2 = "delete from t2"
    sql3 = "delete from t1"
    with engine.connect() as conn:
        conn.execute(text(sql1))
        conn.execute(text(sql2))
        conn.execute(text(sql3))    

## insert_data

In [7]:
def insert_data(engine):
    sql1 = """
    insert into t1 values 
      (1, 'a1', 'b1'), 
      (2, 'a2', 'b2')
    """
    sql2 = """
    insert into t2 values 
      (11, 'c11', 'd11', 1), 
      (12, 'c12', 'd12', 1),
      (13, 'c13', 'd13', 2), 
      (14, 'c14', 'd14', 2),
      (15, 'c15', 'd15', 2)
    """
    sql3 = """
    insert into t3 values 
      (101, 'e101', 'f101', 11), 
      (102, 'e102', 'f102', 11),
      (103, 'e103', 'f103', 12), 
      (104, 'e104', 'f104', 12),
      (105, 'e105', 'f105', 13),
      (106, 'e106', 'f106', 13),
      (107, 'e107', 'f107', 14),
      (108, 'e108', 'f108', 14),
      (109, 'e109', 'f109', 15),
      (110, 'e110', 'f110', 15)
    """
    with engine.connect() as conn:
        conn.execute(text(sql1))
        conn.execute(text(sql2)) 
        conn.execute(text(sql3))

## get_data

In [8]:
def get_data(engine):
    sql1 = "select * from t1"
    sql2 = "select * from t2"
    sql3 = "select * from t3"
    with engine.connect() as conn:
        df1 = pd.read_sql_query(text(sql1), conn)
        df2 = pd.read_sql_query(text(sql2), conn)
        df3 = pd.read_sql_query(text(sql3), conn)
    return df1, df2, df3

## Datenerzeugung

In [9]:
#drop_tables(engine)
create_tables(engine)
# delete_data(engine)
insert_data(engine)

## Datenanzeige

In [10]:
df1, df2, df3 = get_data(engine)

In [11]:
df1

,pk1,a,b
0,1,a1,b1
1,2,a2,b2


In [12]:
df2

,pk2,c,d,fk_t2_t1
0,11,c11,d11,1
1,12,c12,d12,1
2,13,c13,d13,2
3,14,c14,d14,2
4,15,c15,d15,2


In [13]:
df3

,pk3,e,f,fk_t3_t2
0,101,e101,f101,11
1,102,e102,f102,11
2,103,e103,f103,12
3,104,e104,f104,12
4,105,e105,f105,13
5,106,e106,f106,13
6,107,e107,f107,14
7,108,e108,f108,14
8,109,e109,f109,15
9,110,e110,f110,15


# SQLExe

In [14]:
def sqlexe(engine, sql):
    with engine.connect() as conn:
        df = pd.read_sql_query(text(sql), conn)
    return df

# AST

In [15]:
@dataclass
class Cond():
    l: str
    r: str

@dataclass
class AST():
    name: str
    attr: list[str]
    cond: Optional[Cond] = None
    sub: Optional['AST'] = None

# Code zur Lösung der Aufgabenstellung
Bitte in diesem Abschnitt ihren kompletten Code unterbringen. Alle anderen Abschnitte unverändert lassen.

In [26]:
def gql_to_jon(gql: AST) -> str:
    return dummy_json

# Auswertungsbeispiele

## 01

In [28]:
ast01 = AST("t1", ["a"])
print(ast01)
print()
jsonstr = gql_to_jon(ast01)
print(json.dumps(jsonstr, sort_keys=True, indent=4))

AST(name='t1', attr=['a'], cond=None, sub=None)

[
    {
        "dummy": "bisher nicht implemenietiert"
    }
]


## 02

In [33]:
ast02 = AST("t1", ["a", "b"], Cond("a", "a1"))
print(ast02)
print()
jsonstr = gql_to_jon(ast01)
print(json.dumps(jsonstr, sort_keys=True, indent=4))

AST(name='t1', attr=['a', 'b'], cond=Cond(l='a', r='a1'), sub=None)

[
    {
        "dummy": "bisher nicht implemenietiert"
    }
]


## 03

In [32]:
ast03 = AST("t1", ["a", "b"], Cond("a", "a1"), sub=AST("t2", ["c", "d"]))
print(ast03)
print()
jsonstr = gql_to_jon(ast01)
print(json.dumps(jsonstr, sort_keys=True, indent=4))

AST(name='t1', attr=['a', 'b'], cond=Cond(l='a', r='a1'), sub=AST(name='t2', attr=['c', 'd'], cond=None, sub=None))

[
    {
        "dummy": "bisher nicht implemenietiert"
    }
]


## 04

In [31]:
ast04 = AST("t1", ["a", "b"], Cond("a", "a1"), sub=AST("t2", ["c", "d"], Cond("d", "d12")))
print(ast04)
print()
jsonstr = gql_to_jon(ast01)
print(json.dumps(jsonstr, sort_keys=True, indent=4))

AST(name='t1', attr=['a', 'b'], cond=Cond(l='a', r='a1'), sub=AST(name='t2', attr=['c', 'd'], cond=Cond(l='d', r='d12'), sub=None))

[
    {
        "dummy": "bisher nicht implemenietiert"
    }
]


## 05

In [30]:
ast05_t3 = AST("t3", ["e", "f"])
ast05 = AST("t1", ["a", "b"], Cond("a", "a1"), sub=AST("t2", ["c", "d"], Cond("d", "d12"), sub=ast05_t3))
print(ast05)
print()
jsonstr = gql_to_jon(ast01)
print(json.dumps(jsonstr, sort_keys=True, indent=4))

AST(name='t1', attr=['a', 'b'], cond=Cond(l='a', r='a1'), sub=AST(name='t2', attr=['c', 'd'], cond=Cond(l='d', r='d12'), sub=AST(name='t3', attr=['e', 'f'], cond=None, sub=None)))

[
    {
        "dummy": "bisher nicht implemenietiert"
    }
]


## 06

In [29]:
ast06_t3 = AST("t3", ["e", "f"], Cond("f", "f103"))
ast06 = AST("t1", ["a", "b"], Cond("a", "a1"), sub=AST("t2", ["c", "d"], Cond("d", "d12"), sub=ast06_t3))
print(ast06)
print()
jsonstr = gql_to_jon(ast01)
print(json.dumps(jsonstr, sort_keys=True, indent=4))

AST(name='t1', attr=['a', 'b'], cond=Cond(l='a', r='a1'), sub=AST(name='t2', attr=['c', 'd'], cond=Cond(l='d', r='d12'), sub=AST(name='t3', attr=['e', 'f'], cond=Cond(l='f', r='f103'), sub=None)))

[
    {
        "dummy": "bisher nicht implemenietiert"
    }
]
